In [ ]:
import re
import pandas as pd

In [ ]:
f=open('WhatsApp Chat with Pro Megh Ros.txt','r',encoding='utf-8')

In [ ]:
data=f.read()

In [ ]:
print(data)

In [ ]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [ ]:
messages = re.split(pattern,data)[1:]
messages

In [ ]:
dates = re.findall(pattern,data)
dates

In [ ]:
df = pd.DataFrame({'user_message':messages, 'message_date':dates})

In [ ]:
df

In [ ]:
# converting message date-type
df['message_date']=pd.to_datetime(df['message_date'],format='%d/%m/%Y, %H:%M - ')

In [ ]:
df

In [ ]:
#renaming the column
df.rename(columns={'message_date' : 'date'},inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
# separating users and messages

users=[]
messages=[]

for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]: #user_name
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user']=users
df['message']=messages
df.drop(columns=['user_message'],inplace=True)  #removing the prev. column

df.head()

In [ ]:
df['date'].dt.year

In [ ]:
df['year']=df['date'].dt.year

In [ ]:
df

In [ ]:
df['month']=df['date'].dt.month_name()

In [ ]:
df.head()

In [ ]:
df['day']=df['date'].dt.day

In [ ]:
df['hour']=df['date'].dt.hour

In [ ]:
df['minute']=df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
df['user'].unique().tolist()

In [ ]:
df.shape[0]

In [ ]:
df[df['user']=='Projna'].shape

In [ ]:
words=[]
for message in df['message']:
    words.extend(message.split())

In [ ]:
len(words)

In [ ]:
df[df['message']=='<Media omitted>\n'].shape[0]

In [ ]:
pip install urlextract

In [ ]:
from urlextract import URLExtract

In [ ]:
extractor=URLExtract()
urls=extractor.find_urls("hello world www.google.com stackoverflow.com www.youtube.com")
urls

In [ ]:
links=[]
for message in df['message']:
    links.extend(extractor.find_urls(message))
print(links)    

In [ ]:
len(links)

In [ ]:
df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = df['user'].value_counts().head()
name = x.index
count= x.values

plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
round((df['user'].value_counts().head()/df.shape[0])*100,2).reset_index().rename(columns={'user':'name','count':'percent'})

In [ ]:
df['message'].str.cat(sep=" ")

In [ ]:
words = []
for message in df['message']:
    words.extend(message.split())

In [ ]:
words

In [ ]:
pip install Collection

In [ ]:
from collections import Counter
Counter(words)

In [ ]:
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# remove group_notification
# remove media omitted messages
# remove stop words

In [ ]:
temp=df[df['user']!='group_notification']
temp=temp[temp['message']!='<Media omitted>\n']

In [ ]:
f=open('stop_stopwords.txt','r',encoding='utf-8')
stop_words=f.read()
# print(stop_words)

In [ ]:
words = []
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
" ".join(["hello","world","to","all"])

In [ ]:
df['month_num']=df['date'].dt.month

In [ ]:
df.head()

In [ ]:
timeline=df.groupby(['year','month_num','month']).count()['message'].reset_index()
timeline

In [ ]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + '-' + str(timeline['year'][i]))

In [ ]:
timeline['time']=time

In [ ]:
timeline

In [ ]:
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
df['only_date']=df['date'].dt.date

In [ ]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'],daily_timeline['message'])
plt.xticks(rotation='vertical')

In [ ]:
df.head()

In [ ]:
df['day_name']=df['date'].dt.day_name()

In [ ]:
df['day_name'].value_counts()

In [ ]:
type(df['day_name'].value_counts())

In [ ]:
df['month'].value_counts()

In [ ]:
df[['day_name','hour']]

In [ ]:
df[['day_name','hour']]['hour']

In [ ]:
period=[]
for hour in df[['day_name','hour']]['hour']:
    if hour==23:
        period.append(str(hour) + "-" + str("00"))
    elif hour==0:
        period.append(str("00") + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))

In [ ]:
period

In [ ]:
df['period']=period

In [ ]:
df.head()

In [ ]:
import seaborn as sns

In [ ]:
df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0)

In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0))
plt.xticks(rotation='horizontal')
plt.show()